# **Random Forest**
- Using data that was not annotated
- Training it on the unbalanced data
- Using avgpool
- 6 second window

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Standard libraries
import numpy as np
import pandas as pd
import time
import os

# For audio
from IPython.display import Audio
import librosa

# For preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf

# For modeling
from sklearn.metrics import classification_report, roc_auc_score, f1_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

# Operational
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import time
from scipy.signal import butter, filtfilt
import random

In [246]:
pkl_path = '/content/drive/My Drive/Final-Year-Project/Dataset/Final-Version-of-Bird-Classification-Project/feature-extraction/Annotated/Regular/AveragePooled/split_features_3s_all_1D.pkl'

# Load the pickle file
with open(pkl_path, 'rb') as file:
    data = pickle.load(file)

In [247]:
train_data = data['train'].copy()
val_data = data['val'].copy()

In [248]:
train_data

{'melspectrogram': array([[-0.01761791, -0.28131086, -0.26533863, ..., -0.9942341 ,
         -0.99861884, -1.        ],
        [-0.01783594, -0.28790727, -0.27350605, ..., -0.9952871 ,
         -0.999275  , -1.        ],
        [-0.01663349, -0.23677032, -0.2753801 , ..., -0.9895009 ,
         -0.9974265 , -1.        ],
        ...,
        [-0.08988359, -0.4680023 , -0.5110256 , ..., -0.9615641 ,
         -0.9693968 , -0.99108076],
        [-0.0954483 , -0.56289166, -0.5976204 , ..., -0.94434714,
         -0.95767987, -0.99251   ],
        [-0.08350353, -0.4676744 , -0.5083498 , ..., -0.9486351 ,
         -0.9575719 , -0.990598  ]], dtype=float32),
 'mfcc': array([[-0.9992188 ,  0.552929  , -0.3020308 , ...,  0.02957186,
          0.06612163,  0.02512584],
        [-1.        ,  0.54670596, -0.30596924, ...,  0.0395895 ,
          0.06703314,  0.02110985],
        [-0.9954227 ,  0.5823921 , -0.33912998, ...,  0.01689669,
          0.07941275,  0.04006654],
        ...,
        [-1. 

In [249]:
train_labels = train_data['label'].copy()
temp = train_data.copy()
del temp['label']
tr_features = temp

In [250]:
val_labels = val_data['label'].copy()
temp = val_data.copy()
del temp['label']
v_features = temp

# Shuffle Data

In [251]:
def shuffle_data(input_label, input_features):
  input_len = len(input_label)
  np.random.seed(1826)
  input_indices = np.random.permutation(input_len)
  input_features = {key: np.array([input_features[key][i] for i in input_indices]) for key in input_features} # dictionary comprehension
  input_label = np.array([input_label[i] for i in input_indices])

  return input_label, input_features

In [252]:
train_y, train_features = shuffle_data(train_labels, tr_features)

In [253]:
val_y, val_features = shuffle_data(val_labels, v_features)

In [254]:
display(train_y.shape)
display(train_y[:15])

(5278,)

array([ 1, 16,  1,  2, 13,  3, 16, 15,  1,  0,  9, 13,  2, 17,  5],
      dtype=int32)

In [255]:
for key in train_features.keys():
  display(key)
  display(train_features[key].shape)
  display(train_features[key][0])

'melspectrogram'

(5278, 128)

array([-0.01764392, -0.97803795, -0.9799117 , -0.98128027, -0.98216313,
       -0.9831314 , -0.9837664 , -0.98435324, -0.98495674, -0.9852795 ,
       -0.98582494, -0.9861678 , -0.9864845 , -0.9869083 , -0.98708737,
       -0.98747   , -0.9877104 , -0.9879088 , -0.98825246, -0.988359  ,
       -0.9886609 , -0.988563  , -0.9883346 , -0.9867686 , -0.9817328 ,
       -0.9828566 , -0.9821785 , -0.97042674, -0.9548784 , -0.9327689 ,
       -0.9087886 , -0.89279616, -0.8838301 , -0.8622066 , -0.85817975,
       -0.8661405 , -0.860268  , -0.83831847, -0.83264095, -0.8175676 ,
       -0.7988394 , -0.7754159 , -0.764845  , -0.7578966 , -0.7570437 ,
       -0.7563516 , -0.7400953 , -0.72609264, -0.70449096, -0.69452417,
       -0.69010544, -0.66583955, -0.6530399 , -0.6353057 , -0.61711437,
       -0.6123649 , -0.60397524, -0.59884113, -0.59604484, -0.59018356,
       -0.5851924 , -0.5791772 , -0.5721776 , -0.5674804 , -0.5597569 ,
       -0.566274  , -0.58043104, -0.57135457, -0.55910766, -0.55

'mfcc'

(5278, 20)

...

(5278, 84)

array([-65.528885, -66.13044 , -66.251305, -67.008995, -67.87411 ,
       -67.82797 , -68.44748 , -68.9464  , -69.63102 , -69.47168 ,
       -69.87383 , -70.5019  , -73.08208 , -73.35049 , -73.4445  ,
       -73.78593 , -74.243256, -74.1996  , -74.529076, -74.79334 ,
       -74.979355, -75.06892 , -75.31733 , -75.60023 , -76.647804,
       -76.81876 , -76.8498  , -76.95259 , -77.20677 , -77.25853 ,
       -77.40139 , -77.48596 , -77.75067 , -77.80297 , -77.78294 ,
       -77.88078 , -78.350296, -78.3953  , -78.414764, -78.54884 ,
       -78.63048 , -78.629616, -78.703766, -78.51372 , -77.9464  ,
       -77.77281 , -77.75938 , -77.24206 , -75.34778 , -72.82782 ,
       -71.08959 , -69.48764 , -67.96945 , -66.66963 , -63.88107 ,
       -60.09791 , -58.00503 , -56.133953, -54.9986  , -53.39902 ,
       -50.444267, -48.117077, -47.22968 , -45.289127, -43.545864,
       -41.127274, -38.00956 , -35.79028 , -34.706688, -34.253117,
       -33.19932 , -32.110455, -32.374672, -32.6537  , -31.532

'id'

(5278,)

'XC358628.mp3'

In [256]:
display(val_y.shape)
display(val_y[:15])

(1350,)

array([17,  2,  1,  2, 18,  2,  1,  3,  7,  2,  2, 15,  9, 15,  6],
      dtype=int32)

In [257]:
for key in val_features.keys():
  display(key)
  display(val_features[key].shape)
  display(val_features[key][0])

'melspectrogram'

(1350, 128)

array([-0.072525  , -0.5083997 , -0.5475608 , -0.582492  , -0.60023695,
       -0.6163909 , -0.62330556, -0.6467266 , -0.65783316, -0.6554587 ,
       -0.6664068 , -0.67543006, -0.663391  , -0.6618359 , -0.66224116,
       -0.6633849 , -0.6735752 , -0.6648142 , -0.6667964 , -0.6681304 ,
       -0.67080134, -0.6705353 , -0.67797256, -0.6792977 , -0.68008566,
       -0.6855399 , -0.6937607 , -0.6813969 , -0.68022996, -0.668508  ,
       -0.68908864, -0.7102224 , -0.70273346, -0.7106349 , -0.71191967,
       -0.71516126, -0.7085675 , -0.71395266, -0.7190398 , -0.7148571 ,
       -0.7139665 , -0.71852416, -0.71945214, -0.72841054, -0.733488  ,
       -0.7315607 , -0.73327506, -0.73761636, -0.7405483 , -0.7488616 ,
       -0.7493769 , -0.7418681 , -0.75143486, -0.7546518 , -0.7576604 ,
       -0.75809085, -0.7610152 , -0.76253724, -0.7658075 , -0.7666794 ,
       -0.77956283, -0.7787549 , -0.77642643, -0.783159  , -0.7834325 ,
       -0.7959636 , -0.7954486 , -0.804895  , -0.79833907, -0.80

'mfcc'

(1350, 20)

...

(1350, 84)

array([-16.197977, -16.622145, -19.224096, -20.360273, -19.659046,
       -21.36388 , -22.642794, -22.072052, -22.244375, -23.20913 ,
       -24.18649 , -24.797247, -25.787546, -26.665874, -28.718456,
       -29.12966 , -29.949657, -30.821644, -31.857664, -32.350204,
       -31.991999, -33.119484, -32.826878, -33.911182, -35.28518 ,
       -35.572   , -35.864708, -35.425735, -36.176373, -37.05869 ,
       -36.980392, -38.54786 , -39.41071 , -39.10761 , -39.64265 ,
       -38.928444, -38.833355, -38.50526 , -38.857983, -39.006104,
       -37.79628 , -37.00809 , -37.39893 , -36.959846, -37.024754,
       -38.057194, -37.463768, -36.918304, -37.41815 , -37.6202  ,
       -37.67375 , -37.83155 , -38.603985, -38.957176, -38.16948 ,
       -38.03398 , -40.13231 , -40.207188, -40.409916, -40.984615,
       -40.47422 , -41.241203, -41.72562 , -41.77864 , -42.805424,
       -43.701435, -43.36326 , -43.763554, -44.783657, -45.228275,
       -45.982094, -46.685345, -46.98993 , -47.24121 , -48.193

'id'

(1350,)

'XC252756.mp3'

## **Random Forest Model**

Saving evaluation results

In [258]:
def evaluate_model(model, validation_features, val_y):
    # Predict class labels for validation set
    val_yhat_result = model.predict(validation_features)

    # Print classification report
    print('Validation classification Report \n')
    print(classification_report(val_y, val_yhat_result))

    # Get probabilities for the validation set (for AUC calculation)
    val_y_proba = model.predict_proba(validation_features)

    # Calculate AUC for multiclass classification using 'ovr' and 'weighted' average
    auc_score = roc_auc_score(val_y, val_y_proba, multi_class='ovr', average='weighted')
    print(f'AUC Score: {auc_score}')

    # Calculate F1-score with 'weighted' average for imbalanced dataset
    f1 = f1_score(val_y, val_yhat_result, average='weighted')
    print(f'F1 Score (Weighted): {f1}')

    # Store the scores in the dictionary
    val_score = {'f1': f1, 'auc': auc_score}

    # Return the scores dictionary
    return val_score

In [259]:
train_results = {}
val_results = {}

val_scores = {}
k=15

### **With all the features**

In [260]:
training_features = np.concatenate((train_features['mfcc'], train_features['chroma'], train_features['cqt'], train_features['melspectrogram']), axis=1)

training_features.shape

(5278, 244)

In [261]:
validation_features = np.concatenate((val_features['mfcc'], val_features['chroma'], val_features['cqt'], val_features['melspectrogram']), axis=1)

validation_features.shape

(1350, 244)

Fit the model with training data

In [262]:
scaler = StandardScaler()
training_features = scaler.fit_transform(training_features)
validation_features = scaler.transform(validation_features)
model = KNeighborsClassifier(n_neighbors=k, weights='distance')
model.fit(training_features, train_y)

KNeighborsClassifier(n_neighbors=15, weights='distance')

In [263]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['all_features'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['all_features'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.577037037037037


In [264]:
val_scores['all_features'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.53      0.54      0.53       114
           1       0.44      0.64      0.52       141
           2       0.69      0.67      0.68       271
           3       0.94      0.82      0.88        90
           4       0.25      0.06      0.09        18
           5       0.40      0.67      0.50        15
           6       0.13      0.22      0.16        23
           7       0.96      0.98      0.97        44
           8       0.31      0.64      0.42        50
           9       0.85      0.57      0.68        49
          10       0.79      0.48      0.60        48
          11       0.27      0.37      0.31        46
          12       0.71      0.78      0.74        54
          13       0.47      0.52      0.50        48
          14       0.21      0.16      0.18        38
          15       0.27      0.12      0.17       117
          16       0.83      0.80      0.81   

### **With MFCCs only**

In [265]:
training_features = train_features['mfcc']

training_features.shape

(5278, 20)

In [266]:
validation_features = val_features['mfcc']

validation_features.shape

(1350, 20)

In [267]:
scaler = StandardScaler()
training_features = scaler.fit_transform(training_features)
validation_features = scaler.transform(validation_features)
model = KNeighborsClassifier(n_neighbors=k, weights='distance')
model.fit(training_features, train_y)

KNeighborsClassifier(n_neighbors=15, weights='distance')

In [268]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['mfcc'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['mfcc'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.562962962962963


In [269]:
val_scores['mfcc'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.56      0.54      0.55       114
           1       0.33      0.58      0.42       141
           2       0.66      0.47      0.55       271
           3       0.88      1.00      0.94        90
           4       0.00      0.00      0.00        18
           5       0.67      0.13      0.22        15
           6       0.22      0.35      0.27        23
           7       1.00      0.93      0.96        44
           8       0.31      0.56      0.40        50
           9       1.00      0.67      0.80        49
          10       0.40      0.17      0.24        48
          11       0.36      0.30      0.33        46
          12       0.69      0.67      0.68        54
          13       0.44      0.67      0.53        48
          14       0.33      0.68      0.45        38
          15       0.60      0.32      0.42       117
          16       0.78      0.91      0.84   

### **With Mel-Spectrogram Only**

In [270]:
training_features = train_features['melspectrogram']

training_features.shape

(5278, 128)

In [271]:
validation_features = val_features['melspectrogram']

validation_features.shape

(1350, 128)

In [272]:
scaler = StandardScaler()
training_features = scaler.fit_transform(training_features)
validation_features = scaler.transform(validation_features)
model = KNeighborsClassifier(n_neighbors=k, weights='distance')
model.fit(training_features, train_y)

KNeighborsClassifier(n_neighbors=15, weights='distance')

In [273]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['melspectrogram'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['melspectrogram'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.5037037037037037


In [274]:
val_scores['melspectrogram'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.55      0.54      0.54       114
           1       0.33      0.59      0.42       141
           2       0.65      0.45      0.53       271
           3       0.82      0.76      0.79        90
           4       0.17      0.06      0.08        18
           5       0.29      0.60      0.39        15
           6       0.07      0.22      0.10        23
           7       1.00      0.48      0.65        44
           8       0.35      0.60      0.44        50
           9       0.93      0.55      0.69        49
          10       0.45      0.38      0.41        48
          11       0.22      0.15      0.18        46
          12       0.66      0.76      0.71        54
          13       0.62      0.48      0.54        48
          14       0.35      0.45      0.39        38
          15       0.25      0.14      0.18       117
          16       0.84      0.96      0.90   

## **With Chroma Only**

In [275]:
training_features = train_features['chroma']

training_features.shape

(5278, 12)

In [276]:
validation_features = val_features['chroma']

validation_features.shape

(1350, 12)

In [277]:
scaler = StandardScaler()
training_features = scaler.fit_transform(training_features)
validation_features = scaler.transform(validation_features)
model = KNeighborsClassifier(n_neighbors=k, weights='distance')
model.fit(training_features, train_y)

KNeighborsClassifier(n_neighbors=15, weights='distance')

In [278]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['chroma'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['chroma'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.36074074074074075


In [279]:
val_scores['chroma'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.43      0.27      0.33       114
           1       0.27      0.58      0.36       141
           2       0.41      0.22      0.29       271
           3       0.46      0.68      0.55        90
           4       0.00      0.00      0.00        18
           5       0.17      0.07      0.10        15
           6       0.12      0.09      0.10        23
           7       0.54      0.84      0.66        44
           8       0.16      0.18      0.17        50
           9       0.85      0.59      0.70        49
          10       0.33      0.27      0.30        48
          11       0.26      0.28      0.27        46
          12       0.50      0.24      0.33        54
          13       0.31      0.42      0.35        48
          14       0.53      0.42      0.47        38
          15       0.33      0.16      0.22       117
          16       0.40      0.42      0.41   

## **With CQT Only**

In [280]:
training_features = train_features['cqt']

training_features.shape

(5278, 84)

In [281]:
validation_features = val_features['cqt']

validation_features.shape

(1350, 84)

In [282]:
scaler = StandardScaler()
training_features = scaler.fit_transform(training_features)
validation_features = scaler.transform(validation_features)
model = KNeighborsClassifier(n_neighbors=k, weights='distance')
model.fit(training_features, train_y)

KNeighborsClassifier(n_neighbors=15, weights='distance')

In [283]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['cqt'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['cqt'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.4096296296296296


In [284]:
val_scores['cqt'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.53      0.59      0.56       114
           1       0.35      0.55      0.43       141
           2       0.59      0.50      0.54       271
           3       0.20      0.23      0.21        90
           4       0.00      0.00      0.00        18
           5       0.46      0.40      0.43        15
           6       0.12      0.30      0.17        23
           7       0.83      0.91      0.87        44
           8       0.23      0.60      0.34        50
           9       0.83      0.39      0.53        49
          10       0.37      0.31      0.34        48
          11       0.29      0.22      0.25        46
          12       0.61      0.69      0.64        54
          13       0.21      0.12      0.16        48
          14       0.03      0.03      0.03        38
          15       0.19      0.10      0.13       117
          16       0.60      0.80      0.69   

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## **With MFCCs and Mel-Spectrogram**

In [285]:
training_features = np.concatenate((train_features['mfcc'], train_features['melspectrogram']), axis=1)

training_features.shape

(5278, 148)

In [286]:
validation_features = np.concatenate((val_features['mfcc'], val_features['melspectrogram']), axis=1)

validation_features.shape

(1350, 148)

In [287]:
scaler = StandardScaler()
training_features = scaler.fit_transform(training_features)
validation_features = scaler.transform(validation_features)
model = KNeighborsClassifier(n_neighbors=k, weights='distance')
model.fit(training_features, train_y)

KNeighborsClassifier(n_neighbors=15, weights='distance')

In [288]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['mfcc_melspectrogram'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['mfcc_melspectrogram'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.5666666666666667


In [289]:
val_scores['mfcc_melspectrogram'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.51      0.56      0.53       114
           1       0.36      0.60      0.45       141
           2       0.63      0.49      0.55       271
           3       0.95      0.91      0.93        90
           4       0.20      0.06      0.09        18
           5       0.43      0.67      0.53        15
           6       0.10      0.17      0.13        23
           7       0.97      0.77      0.86        44
           8       0.33      0.58      0.42        50
           9       0.93      0.51      0.66        49
          10       0.60      0.38      0.46        48
          11       0.29      0.30      0.29        46
          12       0.71      0.74      0.73        54
          13       0.68      0.71      0.69        48
          14       0.45      0.63      0.53        38
          15       0.40      0.24      0.30       117
          16       0.90      0.95      0.92   

## **With MFCCs and Chroma**

In [290]:
training_features = np.concatenate((train_features['mfcc'], train_features['chroma']), axis=1)

training_features.shape

(5278, 32)

In [291]:
validation_features = np.concatenate((val_features['mfcc'], val_features['chroma']), axis=1)

validation_features.shape

(1350, 32)

In [292]:
scaler = StandardScaler()
training_features = scaler.fit_transform(training_features)
validation_features = scaler.transform(validation_features)
model = KNeighborsClassifier(n_neighbors=k, weights='distance')
model.fit(training_features, train_y)

KNeighborsClassifier(n_neighbors=15, weights='distance')

In [293]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['mfcc_chroma'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['mfcc_chroma'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.5807407407407408


In [294]:
val_scores['mfcc_chroma'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.67      0.55      0.61       114
           1       0.30      0.54      0.39       141
           2       0.62      0.49      0.55       271
           3       0.89      1.00      0.94        90
           4       0.00      0.00      0.00        18
           5       0.67      0.27      0.38        15
           6       0.22      0.26      0.24        23
           7       0.84      0.98      0.91        44
           8       0.37      0.58      0.45        50
           9       0.97      0.76      0.85        49
          10       0.58      0.46      0.51        48
          11       0.53      0.37      0.44        46
          12       0.69      0.44      0.54        54
          13       0.52      0.67      0.58        48
          14       0.39      0.71      0.50        38
          15       0.70      0.37      0.48       117
          16       0.78      0.89      0.83   

## **With MFCCs and CQT**

In [295]:
training_features = np.concatenate((train_features['mfcc'], train_features['cqt']), axis=1)

training_features.shape

(5278, 104)

In [296]:
validation_features = np.concatenate((val_features['mfcc'], val_features['cqt']), axis=1)

validation_features.shape

(1350, 104)

In [297]:
scaler = StandardScaler()
training_features = scaler.fit_transform(training_features)
validation_features = scaler.transform(validation_features)
model = KNeighborsClassifier(n_neighbors=k, weights='distance')
model.fit(training_features, train_y)

KNeighborsClassifier(n_neighbors=15, weights='distance')

In [298]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['mfcc_cqt'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['mfcc_cqt'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.5133333333333333


In [299]:
val_scores['mfcc_cqt'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.66      0.55      0.60       114
           1       0.31      0.54      0.40       141
           2       0.55      0.49      0.52       271
           3       0.76      0.81      0.78        90
           4       0.00      0.00      0.00        18
           5       0.41      0.60      0.49        15
           6       0.24      0.35      0.28        23
           7       0.85      0.93      0.89        44
           8       0.32      0.56      0.41        50
           9       0.90      0.53      0.67        49
          10       0.62      0.38      0.47        48
          11       0.28      0.37      0.32        46
          12       0.73      0.69      0.70        54
          13       0.47      0.60      0.53        48
          14       0.31      0.29      0.30        38
          15       0.38      0.15      0.21       117
          16       0.59      0.78      0.67   

## **With Chroma and CQT**

In [300]:
training_features = np.concatenate((train_features['chroma'], train_features['cqt']), axis=1)

training_features.shape

(5278, 96)

In [301]:
validation_features = np.concatenate((val_features['chroma'], val_features['cqt']), axis=1)

validation_features.shape

(1350, 96)

In [302]:
scaler = StandardScaler()
training_features = scaler.fit_transform(training_features)
validation_features = scaler.transform(validation_features)
model = KNeighborsClassifier(n_neighbors=k, weights='distance')
model.fit(training_features, train_y)

KNeighborsClassifier(n_neighbors=15, weights='distance')

In [303]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['chroma_cqt'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['chroma_cqt'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.482962962962963


In [304]:
val_scores['chroma_cqt'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.58      0.53      0.55       114
           1       0.40      0.53      0.46       141
           2       0.56      0.60      0.58       271
           3       0.47      0.54      0.51        90
           4       0.00      0.00      0.00        18
           5       0.50      0.33      0.40        15
           6       0.20      0.39      0.26        23
           7       0.84      0.95      0.89        44
           8       0.27      0.50      0.35        50
           9       0.86      0.39      0.54        49
          10       0.38      0.42      0.40        48
          11       0.33      0.30      0.31        46
          12       0.74      0.80      0.77        54
          13       0.36      0.29      0.32        48
          14       0.13      0.11      0.12        38
          15       0.10      0.04      0.06       117
          16       0.65      0.76      0.70   

## **With Mel-Spectrogram and Chroma**

In [305]:
training_features = np.concatenate((train_features['melspectrogram'], train_features['chroma']), axis=1)

training_features.shape

(5278, 140)

In [306]:
validation_features = np.concatenate((val_features['melspectrogram'], val_features['chroma']), axis=1)

validation_features.shape

(1350, 140)

In [307]:
scaler = StandardScaler()
training_features = scaler.fit_transform(training_features)
validation_features = scaler.transform(validation_features)
model = KNeighborsClassifier(n_neighbors=k, weights='distance')
model.fit(training_features, train_y)

KNeighborsClassifier(n_neighbors=15, weights='distance')

In [308]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['melspectrogram_chroma'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['melspectrogram_chroma'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.5451851851851852


In [309]:
val_scores['melspectrogram_chroma'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.59      0.56      0.58       114
           1       0.34      0.58      0.43       141
           2       0.62      0.47      0.53       271
           3       0.85      0.84      0.85        90
           4       0.33      0.06      0.10        18
           5       0.31      0.67      0.43        15
           6       0.13      0.26      0.18        23
           7       1.00      0.75      0.86        44
           8       0.34      0.64      0.44        50
           9       0.87      0.69      0.77        49
          10       0.61      0.62      0.62        48
          11       0.29      0.26      0.28        46
          12       0.72      0.70      0.71        54
          13       0.56      0.42      0.48        48
          14       0.36      0.50      0.42        38
          15       0.28      0.15      0.19       117
          16       0.96      0.96      0.96   

## **With Mel-Spectrogram and CQT**

In [310]:
training_features = np.concatenate((train_features['melspectrogram'], train_features['cqt']), axis=1)

training_features.shape

(5278, 212)

In [311]:
validation_features = np.concatenate((val_features['melspectrogram'], val_features['cqt']), axis=1)

validation_features.shape

(1350, 212)

In [312]:
scaler = StandardScaler()
training_features = scaler.fit_transform(training_features)
validation_features = scaler.transform(validation_features)
model = KNeighborsClassifier(n_neighbors=k, weights='distance')
model.fit(training_features, train_y)

KNeighborsClassifier(n_neighbors=15, weights='distance')

In [313]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['melspectrogram_cqt'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['melspectrogram_cqt'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.5244444444444445


In [314]:
val_scores['melspectrogram_cqt'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.51      0.53      0.52       114
           1       0.44      0.63      0.52       141
           2       0.72      0.65      0.68       271
           3       0.83      0.67      0.74        90
           4       0.17      0.06      0.08        18
           5       0.29      0.67      0.40        15
           6       0.04      0.09      0.06        23
           7       0.95      0.93      0.94        44
           8       0.29      0.66      0.40        50
           9       0.88      0.59      0.71        49
          10       0.58      0.46      0.51        48
          11       0.21      0.24      0.22        46
          12       0.73      0.74      0.73        54
          13       0.31      0.23      0.26        48
          14       0.11      0.11      0.11        38
          15       0.12      0.05      0.07       117
          16       0.79      0.80      0.79   

## **With Mel-Spectrogram, MFCCs, CQT**

In [315]:
training_features = np.concatenate((train_features['melspectrogram'], train_features['cqt'], train_features['mfcc']), axis=1)

training_features.shape

(5278, 232)

In [316]:
validation_features = np.concatenate((val_features['melspectrogram'], val_features['cqt'], val_features['mfcc']), axis=1)

validation_features.shape

(1350, 232)

In [317]:
scaler = StandardScaler()
training_features = scaler.fit_transform(training_features)
validation_features = scaler.transform(validation_features)
model = KNeighborsClassifier(n_neighbors=k, weights='distance')
model.fit(training_features, train_y)

KNeighborsClassifier(n_neighbors=15, weights='distance')

In [318]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['melspectrogram_cqt_mfcc'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['melspectrogram_cqt_mfcc'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.5688888888888889


In [319]:
val_scores['melspectrogram_cqt_mfcc'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.54      0.54      0.54       114
           1       0.42      0.62      0.50       141
           2       0.70      0.67      0.68       271
           3       0.95      0.79      0.86        90
           4       0.25      0.06      0.09        18
           5       0.40      0.67      0.50        15
           6       0.12      0.22      0.16        23
           7       0.95      0.95      0.95        44
           8       0.30      0.60      0.40        50
           9       0.90      0.55      0.68        49
          10       0.66      0.40      0.49        48
          11       0.24      0.30      0.27        46
          12       0.69      0.76      0.73        54
          13       0.46      0.58      0.51        48
          14       0.25      0.18      0.21        38
          15       0.27      0.13      0.17       117
          16       0.80      0.80      0.80   

## **With Mel-Spectrogram, MFCCs, Chroma**

In [320]:
training_features = np.concatenate((train_features['melspectrogram'], train_features['chroma'], train_features['mfcc']), axis=1)

training_features.shape

(5278, 160)

In [321]:
validation_features = np.concatenate((val_features['melspectrogram'], val_features['chroma'], val_features['mfcc']), axis=1)

validation_features.shape

(1350, 160)

In [322]:
scaler = StandardScaler()
training_features = scaler.fit_transform(training_features)
validation_features = scaler.transform(validation_features)
model = KNeighborsClassifier(n_neighbors=k, weights='distance')
model.fit(training_features, train_y)

KNeighborsClassifier(n_neighbors=15, weights='distance')

In [323]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['melspectrogram_chroma_mfcc'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['melspectrogram_chroma_mfcc'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.5955555555555555


In [324]:
val_scores['melspectrogram_cqt_mfcc'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.57      0.57      0.57       114
           1       0.36      0.58      0.45       141
           2       0.62      0.53      0.57       271
           3       0.93      0.93      0.93        90
           4       0.25      0.06      0.09        18
           5       0.47      0.60      0.53        15
           6       0.16      0.22      0.18        23
           7       1.00      0.93      0.96        44
           8       0.34      0.66      0.45        50
           9       0.94      0.63      0.76        49
          10       0.74      0.52      0.61        48
          11       0.42      0.33      0.37        46
          12       0.71      0.78      0.74        54
          13       0.67      0.71      0.69        48
          14       0.44      0.61      0.51        38
          15       0.41      0.22      0.29       117
          16       0.90      0.95      0.92   

# Review results from all models

In [325]:
train_results_df = pd.DataFrame(list(train_results.items()), columns=['Features', 'Train_Accuracy']).round(3)
val_results_df = pd.DataFrame(list(val_results.items()), columns=['Features', 'Val_Accuracy']).round(3)

result_df = train_results_df.merge(val_results_df, on='Features')
result_df = result_df.sort_values('Features')
result_df

,Features,Train_Accuracy,Val_Accuracy
0,all_features,1.0,0.577
3,chroma,1.0,0.361
8,chroma_cqt,1.0,0.483
4,cqt,1.0,0.410
2,melspectrogram,1.0,0.504
9,melspectrogram_chroma,1.0,0.545
12,melspectrogram_chroma_mfcc,1.0,0.596
10,melspectrogram_cqt,1.0,0.524
11,melspectrogram_cqt_mfcc,1.0,0.569
1,mfcc,1.0,0.563


In [326]:
val_scores_df = pd.DataFrame([(key, value['f1'], value['auc']) for key, value in val_scores.items()],
                             columns=['Features', 'F1_Score', 'AUC_Score']).round(3)

val_scores_df = val_scores_df.sort_values('Features')
print(val_scores_df)

                   Features  F1_Score  AUC_Score
0              all_features     0.572      0.902
3                    chroma     0.347      0.822
8                chroma_cqt     0.469      0.862
4                       cqt     0.396      0.831
2            melspectrogram     0.509      0.877
9     melspectrogram_chroma     0.547      0.892
10       melspectrogram_cqt     0.523      0.873
11  melspectrogram_cqt_mfcc     0.595      0.901
1                      mfcc     0.558      0.892
6               mfcc_chroma     0.579      0.903
7                  mfcc_cqt     0.506      0.867
5       mfcc_melspectrogram     0.569      0.894
